In [2]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [4]:
ROOT_DIR = "data"
RANDOM_STATE = 110

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
train_data


,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 464 entries, Wip Line_Dam to target
dtypes: float64(350), int64(77), object(37)
memory usage: 143.4+ MB


In [6]:
train_data.shape[0]

40506

In [7]:
df = train_data.copy()

In [8]:
# 각 열별 결측치 개수 확인
missing_values_count = (df.isnull().sum())
missing_values_count = missing_values_count/df.shape[0] 
missing_values_ratio = missing_values_count * 100
print(missing_values_ratio)

Wip Line_Dam                       0.0
Process Desc._Dam                  0.0
Equipment_Dam                      0.0
Model.Suffix_Dam                   0.0
Workorder_Dam                      0.0
                                 ...  
Receip No Judge Value_Fill2      100.0
WorkMode Collect Result_Fill2      0.0
WorkMode Unit Time_Fill2         100.0
WorkMode Judge Value_Fill2       100.0
target                             0.0
Length: 464, dtype: float64


In [9]:
bins = [0, 20, 40, 60, 80, 100]
missing_values_counts = pd.cut(missing_values_ratio, bins=bins).value_counts().sort_index()

print(missing_values_counts)

(0, 20]        0
(20, 40]       3
(40, 60]       0
(60, 80]       5
(80, 100]    278
Name: count, dtype: int64


(20, 40]에 대해서는 결측치를 채우는 과정을 수행한다.

In [10]:
# 결측치 비율이 80 이상인 열 삭제
threshold = 80
df= df.drop(columns=missing_values_ratio[missing_values_ratio >= threshold].index)

df.info

<bound method DataFrame.info of       Wip Line_Dam Process Desc._Dam     Equipment_Dam Model.Suffix_Dam  \
0          IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334505   
1          IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334505   
2          IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
3          IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
4          IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   
...            ...               ...               ...              ...   
40501      IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   
40502      IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
40503      IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   
40504      IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
40505      IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   

      Workorder_Dam  Insp. Seq No._Dam Insp Judge Code_Dam  \
0    

## 데이터 탐색

칼럼들을 각 공정 단계 별로 구별하여 집중해서 보도록 한다.

In [11]:
# "Process Desc"를 포함하는 열의 인덱스 찾기
process_columns = [col for col in df.columns if "Process Desc" in col]
process_indices = [df.columns.get_loc(col) for col in process_columns]

# 새로운 DataFrame을 저장할 딕셔너리 초기화
dfs = {}

# Process Desc에 따라 DataFrame을 나누어 저장
for i, start_idx in enumerate(process_indices):
    end_idx = process_indices[i + 1]-1 if i + 1 < len(process_indices) else len(df.columns)-1
    
    # 각 Process Desc와 그에 해당하는 환경 변수들을 포함하는 DataFrame 생성
    relevant_columns = df.columns[start_idx:end_idx]
    dfs[f'df{i+1}'] = df[relevant_columns]

# 나눠진 DataFrame들을 확인
for name, df_subset in dfs.items():
    print(f'{name}:')
    print(df_subset)
    print('\n')

df1:
      Process Desc._Dam     Equipment_Dam Model.Suffix_Dam Workorder_Dam  \
0         Dam Dispenser  Dam dispenser #1      AJX75334505    4F1XA938-1   
1         Dam Dispenser  Dam dispenser #1      AJX75334505    3KPM0016-2   
2         Dam Dispenser  Dam dispenser #2      AJX75334501    4E1X9167-1   
3         Dam Dispenser  Dam dispenser #2      AJX75334501    3K1X0057-1   
4         Dam Dispenser  Dam dispenser #1      AJX75334501    3HPM0007-1   
...                 ...               ...              ...           ...   
40501     Dam Dispenser  Dam dispenser #1      AJX75334501    3J1XF434-2   
40502     Dam Dispenser  Dam dispenser #2      AJX75334501    4E1XC796-1   
40503     Dam Dispenser  Dam dispenser #1      AJX75334501    4C1XD438-1   
40504     Dam Dispenser  Dam dispenser #2      AJX75334501    3I1XA258-1   
40505     Dam Dispenser  Dam dispenser #1      AJX75334501    3G1XA501-1   

       Insp. Seq No._Dam Insp Judge Code_Dam  \
0                      1          

In [12]:
df1 = dfs['df1']
df2 = dfs['df2']
df3 = dfs['df3']
df4 = dfs['df4']

상수 변수 탐지 : 모든 항이 동일한 상수 변수의 경우 가치가 존재하지 않는다.

In [13]:
# 상수 열 찾기
constant_columns1 = [col for col in df1.columns if df1[col].nunique() == 1]
constant_columns1

['Process Desc._Dam',
 'Insp. Seq No._Dam',
 'Insp Judge Code_Dam',
 'CURE STANDBY POSITION X Collect Result_Dam',
 'CURE STANDBY POSITION Z Collect Result_Dam',
 'CURE STANDBY POSITION Θ Collect Result_Dam',
 'CURE START POSITION Z Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam']

In [14]:
# 상수 열 삭제
df1_cleaned = df1.drop(columns=constant_columns1)
print(df1_cleaned.shape[1])
print(df1.shape[1])

69
77


In [15]:
def remove_constant_columns(df):
    """
    DataFrame에서 상수 열(모든 값이 동일한 열)을 찾아 삭제하고, 삭제된 열의 이름을 반환합니다.
    
    Parameters:
    df (pd.DataFrame): 상수 열을 삭제할 DataFrame

    Returns:
    tuple: (상수 열이 제거된 DataFrame, 삭제된 상수 열의 리스트)
    """
    # 상수 열 찾기
    constant_columns = [col for col in df.columns if df[col].nunique() == 1]
    
    # 상수 열 출력
    print("삭제된 상수 열:", constant_columns)
    
    # 상수 열 삭제
    df_cleaned = df.drop(columns=constant_columns)
    
    return df_cleaned, constant_columns

In [16]:
df2_cleaned, constant_columns2 = remove_constant_columns(df2)
df3_cleaned, constant_columns3 = remove_constant_columns(df3)
df4_cleaned, constant_columns4 = remove_constant_columns(df4)

삭제된 상수 열: ['Process Desc._AutoClave', 'Equipment_AutoClave', 'Insp. Seq No._AutoClave', 'Insp Judge Code_AutoClave', '1st Pressure Judge Value_AutoClave', '2nd Pressure Judge Value_AutoClave', '3rd Pressure Judge Value_AutoClave', 'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave', 'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave']
삭제된 상수 열: ['Process Desc._Fill1', 'Insp. Seq No._Fill1', 'Insp Judge Code_Fill1', 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1']
삭제된 상수 열: ['Process Desc._Fill2', 'Insp. Seq No._Fill2', 'Insp Judge Code_Fill2', 'CURE END POSITION Θ Collect Result_Fill2', 'CURE STANDBY POSITION X Collect Result_Fill2', 'CURE STANDBY POSITION Θ Collect Result_Fill2', 'CURE START POSITION Θ Collect Result_Fill2', 'DISCHARGED SPEED OF RESIN Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill2', 'Dispense Volume(Stage1) Colle

# 중복값 확인 및 제거

공정 단계 별로 봤을 때 중복값이 존재한다.

In [23]:
df.shape

(40506, 186)

In [22]:
print(df.duplicated(keep = False).sum())

0


In [17]:
print(df1_cleaned.duplicated(keep = False).sum())

0


In [18]:
print(df2_cleaned.duplicated(keep = False).sum())

39820


In [20]:
print(df3_cleaned.duplicated(keep = False).sum())

2


In [21]:
print(df4_cleaned.duplicated(keep = False).sum())

2


## 피처 줄이기를 위한 상관 관계 및 의미 분석.
df2는 빼고 진행한다.
좌표를 합칠 수 있다면 좋아 보이기도 함.

In [24]:
df1_cleaned.columns

Index(['Equipment_Dam', 'Model.Suffix_Dam', 'Workorder_Dam',
       'CURE END POSITION X Collect Result_Dam',
       'CURE END POSITION Z Collect Result_Dam',
       'CURE END POSITION Θ Collect Result_Dam',
       'CURE SPEED Collect Result_Dam',
       'CURE START POSITION X Collect Result_Dam',
       'CURE START POSITION Θ Collect Result_Dam',
       'DISCHARGED SPEED OF RESIN Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam',
       'Dispense Volume(Stage1) Collect Result_Dam',
       'Dispense Volume(Stage2) Collect Result_Dam',
       'Dispense Volume(Stage3) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
       'HEAD NORMAL COORDINATE Y AXIS(Stage1) Col

In [29]:
# 새로운 칼럼 생성
df['CURE POSITION (X, Z, Θ)'] = list(zip(df['CURE END POSITION X Collect Result_Dam'],
                                         df['CURE END POSITION Z Collect Result_Dam'],
                                         df['CURE END POSITION Θ Collect Result_Dam'],
                                        df['CURE START POSITION X Collect Result_Dam'],
                                        df['CURE START POSITION Θ Collect Result_Dam']))

In [30]:
# unique 값의 개수 출력
unique_count = df['CURE POSITION (X, Z, Θ)'].nunique()
print(f"Unique 값의 개수: {unique_count}")

Unique 값의 개수: 2


이진 분류가 가능한 항목이다.  
총 4개의 x,y,z,theta 조합이 있을 때 1,2,3,4  
cure 위치는 항상 1->3 or 2->4이다.  
cure 속도와 연합하면 도움이 될까? cure 시간을 도출할 수 있어 보임.

X, Z, Θ의 종류는 하나밖에 없다.

Equipment_Dam: 사용된 장비의 식별자.  
Model.Suffix_Dam: 모델
의 접미사.  
Workorder_Dam: 작업 지시서 번호.  
CURE END POSITION X/Z/Θ Collect Result_Dam: 경화 종료 위치의 X, Z, Θ 좌표.  
CURE SPEED Collect Result_Dam: 경화 속도.  
CURE START POSITION X/Θ Collect Result_Dam: 경화 시작 위치의 X, Θ 좌표.  
DISCHARGED SPEED OF RESIN Collect Result_Dam: 레진 방출 속도.  
DISCHARGED TIME OF RESIN(Stage1/2/3) Collect Result_Dam: 각 단계에서의 레진 방출 시간.



Dispense Volume(Stage1/2/3) Collect Result_Dam: 각 단계에서의 분배량.


HEAD NORMAL COORDINATE X/Y/Z AXIS(Stage1/2/3) Collect Result_Dam: 각 단계에서의 헤드 정상 좌표.
HEAD Standby Position X/Y/Z Collect Result_Dam: 헤드 대기 위치의 X, Y, Z 좌표.
Head Clean Position X/Y/Z Collect Result_Dam: 헤드 청소 위치의 X, Y, Z 좌표.
Head Purge Position X/Y/Z Collect Result_Dam: 헤드 퍼지 위치의 X, Y, Z 좌표.
Head Zero Position X/Y/Z Collect Result_Dam: 헤드 제로 위치의 X, Y, Z 좌표.
Machine Tact time Collect Result_Dam: 기계 택트 타임.
PalletID Collect Result_Dam: 팔레트 식별자.
Production Qty Collect Result_Dam: 생산 수량.
Receip No Collect Result_Dam: 영수증 번호.
Stage1/2/3 Circle1/2/3/4 Distance Speed Collect Result_Dam: 각 단계와 원의 거리 속도.
Stage1/2/3 Line1/2/3/4 Distance Speed Collect Result_Dam: 각 단계와 선의 거리 속도.
THICKNESS 1/2/3 Collect Result_Dam: 두께 측정값.
WorkMode Collect Result_Dam: 작업 모드.